In [1]:
import numpy as np
from astropy.coordinates import Angle
from astropy.io import fits
from astropy import units as u 
from ants import absInt, fluxInt, headPlay, fitsPlay #radiobs modules


fl = fluxInt.fluxint()
hp = headPlay.headplay()
fp = fitsPlay.fitsplay()


get_ipython().magic(u'pylab inline')


home = '/Users/'
home = '/home/'


rootDir = home+'maccagni/Projects/MFS/FornaxA/centreHI/'
cubeDir = home+'maccagni/data/MFS/MeerKATComm/fa4/cubes/'
acaDir = home+'maccagni/data/MFS/FornaxA/ACA/'

print('''\n\t+---------+\n\t Inputs loaded\n\t+---------+''')

Populating the interactive namespace from numpy and matplotlib

	+---------+
	 Inputs loaded
	+---------+


### Convert mom0 to map in column density units

$$
N_{\rm HI} = 3.1\times10^{17} \frac{SdV}{\Theta^2}
$$

- SdV : integrated flux in **Jy m s$^{-1}$** or **mJy km s$^{-1}$**
- $\Theta^2$ : beam size in **arcminutes**

In [2]:
#------------------#
# 20 asec HI
#------------------#

momDirIn = rootDir+'inMoms/20asec/'
momDirOut = rootDir+'moments/'
mom0Name= momDirIn+'MIAPP-t10_6_mom0.fits'

nHIName=momDirOut+'mom0_NHI_20asec.fits'

momHead = hp.printHead(mom0Name)

print '''\n\t+---------+\n\t Check BUNIT and modify conversion factor accordingly\n\t+---------+'''

SIMPLE	True
BITPIX	-32
NAXIS	2
NAXIS1	25
NAXIS2	34
CTYPE1	RA---SIN
CRPIX1	14.0
CDELT1	-0.00138888888889
CRVAL1	50.67375
CTYPE2	DEC--SIN
CRPIX2	18.0
CDELT2	0.00138888888889
CRVAL2	-37.2083333333
ORIGIN	SoFiA 2.0.0
SPECSYS	0.0
EQUINOX	2000.0
BUNIT	Jy/beam*m/s
BMAJ	0.00664442496252
BMIN	0.00533499983768
BPA	134.940537921

	+---------+
	 Check BUNIT and modify conversion factor accordingly
	+---------+


In [ ]:
bMaj = Angle(momHead['BMAJ'], u.deg)
bMin = Angle(momHead['BMIN'], u.deg)

conversionFactor = 3.1e17/(bMaj.arcminute*bMin.arcminute)
print conversionFactor

#multiply by conversion factor
fp.multVal(mom0Name,conversionFactor,nHIName)
hp.putHead(nHIName,'BUNIT','atoms cm-2')

print '''\n\t+---------+\n\t Conversion to NHI done\n\t+---------+'''

In [ ]:
#------------------#
# 10 asec HI
#------------------#

momDirIn = rootDir+'inMoms/10asec/'
momDirOut = rootDir+'moments/'
mom0Name= momDirIn+'fa4_unf_bin35_th20_dil3_5_mom0.fits'

nHIName=momDirOut+'mom0_NHI_10asec.fits'
print conversionFactor

momHead = hp.printHead(mom0Name)

print '''\n\t+---------+\n\t Check BUNIT and modify conversion factor accordingly\n\t+---------+'''


In [ ]:
bMaj = Angle(momHead['BMAJ'], u.deg)
bMin = Angle(momHead['BMIN'], u.deg)

conversionFactor = 3.1e17*1e3/(bMaj.arcminute*bMin.arcminute)
print conversionFactor
#multiply by conversion factor
fp.multVal(mom0Name,conversionFactor,nHIName)
hp.putHead(nHIName,'BUNIT','atoms cm-2')
print '''\n\t+---------+\n\t Conversion to NHI done\n\t+---------+'''

## Convert to same units

- **HI**
  - the mom0 map at 10 asec is in units of **Jy beam$^{-1} \cdot$ km s$^{-1}$**. Convert to **Jy beam$^{-1}\cdot$  m s$^{-1}$** for conformity with mom0 map at 20 asec.
    - why does this happen? these mom0 are ouputs of `SoFiA` from cubes that were made with `MeerKATHI`. The only difference should be the tapering.
- **CO**
  - cube is in **Jy beam$^{-1}\cdot$ Hz**. Convert to **radio velocity** (**VRAD**) for conformity with HI cubes.
  - BUNIT of mom0 map is in **Jy beam$^{-1}\cdot$ Hz**, but the real units (as in the paper and as HI moment maps) are **Jy beam$^{-1}\cdot$  m s$^{-1}$**. Need to divide mom0 map by CDELT3 of CO_vrad.fits datacube.
      


### Line frequencies and velocity conversions
----

The sky frequency ($\nu$) at which we must observe a spectral line is derived from the rest frequency of the spectral line ($\nu_0$), the line-of-sight velocity of the source ($V$), and the speed of light ($c$). The **relativistic velocity**, or **true line-of-sight velocity**, is related to the observed and rest frequencies by:
  
$$
V = \frac{\nu_0^2-\nu^2}{\nu_0^2+\nu^2}c
$$

in astronomy two different approximations are typically used:

**Radio velocity (VRAD)**

$$
V_{\rm radio} = \frac{\nu_0^2-\nu^2}{\nu_0^2}c = \frac{\lambda^2-\lambda_0^2}{\lambda^2}c 
$$

**Optical velocity (VOPT)**

$$
V_{\rm opt} = \frac{\lambda^2-\lambda_0^2}{\lambda_0^2}c = cz
$$

 

The radio and optical velocities are not identical **$V_{\rm opt}\neq V_{\rm radio}$** Particularly, $V_{\rm optical}$ and $V_{\rm radio}$ diverge for large velocities. 

Note that the natural spectral axis of radio interferometers is in frequency. The radio convention is a velocity relabeling to the frequency axis. Using the optical velocity and redshift, however, introduces a non-linearity between channel widths and labeling, in particular for large velocity values.

----
**NRAO**: https://science.nrao.edu/facilities/vla/docs/manuals/obsguide/modes/line


In [ ]:
#------------------#
# 10 asec HI
#------------------#

momDirIn = rootDir+'inMoms/10asec/'
momDirOut = rootDir+'moments/'
mom0Name= momDirIn+'fa4_unf_bin35_th20_dil3_5_mom0.fits'

mom0msName=momDirOut+'mom0_10asec.fits'

#multiply by conversion factor
fp.multVal(mom0Name,1e3,mom0msName)
hp.putHead(mom0msName,'BUNIT','Jy/beam*m/s')


print '''\n\t+---------+\n\t Units Converted\n\t+---------+'''


In [ ]:
#------------------#
# CO to VRAD
#------------------#
coName = acaDir+'NGC1316_TP_7M_CO.feather.image.fits'

coNameOut = rootDir+'cubes/CO_vrad.fits'
coHead = hp.freqToVrad(coName,coNameOut)

print '''\n\t+---------+\n\t Cube Converted to VRAD\n\t+---------+'''



In [ ]:
#------------------#
# CO BUNIT
#------------------#

coMomName = acaDir+'NGC1316_TP_7M_CO.feather.image_mom0.fits.gz'
momDirOut = rootDir+'moments/'

momOutName=momDirOut+'mom0_CO.fits'

fileIn = fits.open(coMomName)
datas = fileIn[0].data
heads  = fileIn[0].header
datas = np.divide(datas,-float(coHead['CDELT3']))
heads['BUNIT'] = 'Jy/beam*m/s'
del heads['DATAMIN']
del heads['DATAMAX']
fits.writeto(momOutName,datas,heads,overwrite=True)

print '''\n\t+---------+\n\t BUNIT modified\n\t+---------+'''
